## RQ1 Creating FAO forest map

[Add Description]

Need to create a FAO Definition Approximation following the steps from Johnson et al (2023)

Steps:
1. Data Preparation
    1. ~~Clip the 5 GER LULC shapefiles to the Natura 2000 areas~~ (SKIP FOR NOW)
    2. Merge the 2 required GER LULC shapefiles into 1 shapefile
    3. vectorise the 5m JAXA FNF raster
2. Intersect JAXA and GER LULC shapefiles
3. Conditional Reclassing
4. Convert to 5m raster

In [1]:
# SETUP

# Note: this .ipynb file depends on files & folder structures created in rq1_step1_data_prep.ipynb

# Import packages
import glob
import os
import subprocess

import pandas as pd
import geopandas as gpd

#from osgeo import gdal


### Step 1: Data Preparation

#### Step 1.1: Clip GER LULC SHPs

~~In the rq1_step1_data_prep.ipynb file, I clipped all the output **rasters** to the Germany Natura 2000 areas - in this first data prep step, I do the same for the GER LULC shapefiles, as I will be working with vector data for creating the FAO-aligned forest map. ~~

IMPORTANT: Skipping this clipping step for now as I didn't actually manage to clip the output rasters. 

In [3]:
# CLIP GER LULC SHPS

# Store paths to reprojected GER LULC SHPs in a list
#ger_lulc_paths = glob.glob('./processing/clc5_class*xx_3035_DE.shp')

# Create a function which which clips the shp to the Germany Natura areas (& saves to processing folder)
#def clip_shp_to_natura(input_paths):
#    # Load Germany Natura 2000 areas
#    natura_de_gdf = gpd.read_file("./outputs/natura2000_3035_DE.shp")
    # Iterate through the GER LULC shp paths 
#    for path in input_paths:
        # Open the shp for each path 
#        ger_lulc_shp = gpd.read_file(path)
        
        # Clip input GER LULC shp to Natura shp
#        shp_clip  = gpd.clip(ger_lulc_shp, natura_de_gdf)

        # For output file naming: extract the input file name (with extension)
#        name_w_ext = os.path.split(path)[1] 
        # For output file naming: remove extension from input file name 
#        name_wo_ext = os.path.splitext(name_w_ext)[0]
        # For output file naming: create the new name for clipped shp
#        new_name = name_wo_ext + "_clipped.shp"

        # Write the reprojected shp to the processing folder
#        shp_clip.to_file('./processing/' + new_name)

# Run the function for the German LULC zipped shps
#clip_shp_to_natura(ger_lulc_paths)


#### Step 1.2: Merge GER LULC to 1 shp 

In order to make the next steps easier, all the GER LULC shps can be merged into 1 master shp. 

Help for merging/appending shps: https://geopandas.org/en/stable/docs/user_guide/mergingdata.html 

In [ ]:
# MERGE GER LULC SHPS

# Load all the GER LULC SHPs
ger_lulc_class3_shp = gpd.read_file("./processing/clc5_class3xx_3035_DE.shp")
ger_lulc_class4_shp = gpd.read_file("./processing/clc5_class4xx_3035_DE.shp")

# Append the shapefiles together
merged_ger_lulc_shp = pd.concat([ger_lulc_class3_shp,
                                 ger_lulc_class4_shp,
                                 ])

# Check outputs
#print(merged_ger_lulc_shp[1:20])

# Write the merged output to file
merged_ger_lulc_shp.to_file('./processing/clc5_class3_class4_3035_DE.shp')

c:\Users\ninam\Documents\UZH\04_Thesis\code\nm_forest_thesis\thesis_env_conda\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


   CLC18                                           geometry
1    311  POLYGON ((4344570.571 2691193.334, 4344574.302...
2    311  POLYGON ((4344262.931 2692187.589, 4344262.643...
3    311  POLYGON ((4339509.672 2692518.025, 4339513.666...
4    311  POLYGON ((4341026.689 2693014.154, 4341025.65 ...
5    311  POLYGON ((4344382.151 2693173.218, 4344386.392...
6    311  POLYGON ((4343928.217 2694117.347, 4343931.461...
7    311  POLYGON ((4343513.682 2694247.781, 4343520.605...
8    311  POLYGON ((4341169.544 2694691.741, 4341192.809...
9    311  POLYGON ((4339452.012 2694836.674, 4339482.499...
10   311  POLYGON ((4343542.631 2694886.932, 4343549.843...
11   311  POLYGON ((4340025.337 2695106.069, 4340029.588...
12   311  POLYGON ((4343398.117 2695650.86, 4343401.723 ...
13   311  POLYGON ((4341092.109 2695993.755, 4341095.105...
14   311  POLYGON ((4342999.07 2696141.094, 4343002.811 ...
15   311  POLYGON ((4343775.375 2696490.163, 4343775.054...
16   311  POLYGON ((4341589.28 2696844.4

#### Step 1.3: vectorise clipped Jaxa

In order to implement the workflow for creating the FAO-aligned forest map, all input data must be vectors. In this step I vectorise the clipped JAXA 5m raster.

Help with raster to vector: https://py.geocompx.org/05-raster-vector



In [ ]:
# POLYGONISE JAXA

input2 = "./processing/jaxa_FNF_3035_DE_5m_reclass.tif"
gdal_polygonize = "./thesis_env_conda/Scripts/gdal_polygonize.exe"
jaxa_poly = "./processing/jaxa_FNF_3035_DE_5m_reclass.shp"

test2 = subprocess.run([gdal_polygonize, input2, jaxa_poly],
                     capture_output=True, 
                     text=True)

print(test2.stdout)
print(test2.stderr)